# Proxies for spike shapes for Hay model

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import neuron   
import LFPy
import scipy.fftpack as ff
from brainsignals.plotting_convention import simplify_axes
from brainsignals.neural_simulations import return_hay_cell
import brainsignals.neural_simulations as ns
h = neuron.h

np.random.seed(12345)
ns.load_mechs_from_folder(ns.cell_models_folder)

In [ ]:
def insert_current_stimuli(cell):
    stim_params = {'amp': -0.4,
                   'idx': 0,
                   'pptype': "ISyn",
                   'dur': 1e9,
                   'delay': 0}
    synapse = LFPy.StimIntElectrode(cell, **stim_params)
    return synapse, cell

In [ ]:
dt = 2**-6
tstop = 150

# Time window to extract spike from:
t0 = 118
t1 = 123

cell = return_hay_cell(tstop=tstop, dt=dt, make_passive=False)
ns.point_axon_down(cell)
syn, cell = insert_current_stimuli(cell)
cell.simulate(rec_imem=True, rec_vmem=True)
t0_idx = np.argmin(np.abs(cell.tvec - t0))
t1_idx = np.argmin(np.abs(cell.tvec - t1))

cell.vmem = cell.vmem[:, t0_idx:t1_idx]
cell.imem = cell.imem[:, t0_idx:t1_idx]
cell.tvec = cell.tvec[t0_idx:t1_idx] - cell.tvec[t0_idx]


In [ ]:
elec_params = dict(
            sigma = 0.3,      # extracellular conductivity
            x = np.array([0, 100, 200, 1000]),
            y = np.zeros(4),
            z = np.zeros(4),
            method = 'root_as_point',
        )

elec = LFPy.RecExtElectrode(cell, **elec_params)
M_elec = elec.get_transformation_matrix()
eaps = M_elec @ cell.imem * 1000

vmem = cell.vmem
tvec = cell.tvec

In [ ]:
print(cell.d[0]/2)
vmem_shifted = vmem[0, :] - np.linspace(vmem[0, 0], vmem[0, -1], vmem.shape[1])
eaps[0] -= eaps[0, 0]
eaps[1] -= eaps[1, 0]

somav_fft = ff.fft(vmem_shifted)
sample_freq = ff.fftfreq(vmem_shifted.shape[0], d=dt / 1000)
eap_prox = -ff.ifft(np.emath.sqrt(1j * sample_freq) * somav_fft)

plt.close("all")
fig = plt.figure(figsize=[6, 2.3])
fig.subplots_adjust(wspace=0.2, left=0.1, right=0.98, bottom=0.15, top=0.97)

ax2 = fig.add_subplot(111, xlim=[-1, 2], ylabel="normalized", xlabel="time (ms)")

l1, = ax2.plot(tvec - tvec[np.argmax(np.abs(eaps[0]))], eaps[0] / np.max(np.abs(eaps[0])), 'k', lw=2)
l2, = ax2.plot(tvec - tvec[np.argmax(np.abs(eaps[1]))], eaps[1] / np.max(np.abs(eaps[1])), 'k--', lw=2)
l3, = ax2.plot(tvec - tvec[np.argmax(np.abs(eap_prox))], np.real(eap_prox) / np.max(np.abs(eap_prox)), c='r', lw=1)
l4, = ax2.plot(tvec[1:] - tvec[np.argmax(np.abs(np.diff(vmem[0])))], -np.diff(vmem[0]) / np.max(np.abs(np.diff(vmem[0]))), c='b', lw=1)
l5, = ax2.plot(tvec - tvec[np.argmax(np.abs(vmem_shifted))], -vmem_shifted / np.max(np.abs(vmem_shifted)), c='gray', lw=1)

fig.legend([l1, l2,
            l3, l4, l5], [r"$V_{\rm e}$ (6.7 µm)", r"$V_{\rm e}$ (100 µm)",
                                  r"$\sqrt{f}\hat{V}_{\rm m,s}(f)$", 
                          r"$dV_{\rm m,s}/dt$", r"-$V_{\rm m,s}$"], frameon=False, 
           loc=(0.63, 0.32), ncol=2)

simplify_axes(fig.axes)
fig.savefig("spike_proxies_hay.pdf")